In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [2]:

data = pd.read_csv('processed/preprocessed_cv_data.csv')
data['accepted'] = 1

In [3]:
X = data['processed_text'].values
y = data['accepted'].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Tokenize the input text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)


In [5]:
# Tokenize the input text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [6]:
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

In [7]:
vocab_size = len(tokenizer.word_index) + 1

# Pad the sequences to ensure equal length
max_sequence_length = max(len(seq) for seq in X_train_tokens)

X_train_pad = pad_sequences(X_train_tokens, maxlen=max_sequence_length, padding='post')
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_sequence_length, padding='post')


In [8]:
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=max_sequence_length))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [9]:
# Train the model
model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=10, batch_size=32)


Epoch 1/10
6/6 [==============================] - 13s 820ms/step - loss: 0.4253 - accuracy: 1.0000 - val_loss: 0.0462 - val_accuracy: 1.0000
Epoch 2/10
6/6 [==============================] - 4s 646ms/step - loss: 0.0296 - accuracy: 1.0000 - val_loss: 0.0072 - val_accuracy: 1.0000
Epoch 3/10
6/6 [==============================] - 4s 669ms/step - loss: 0.0062 - accuracy: 1.0000 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 4/10
6/6 [==============================] - 4s 641ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 6.8152e-04 - val_accuracy: 1.0000
Epoch 5/10
6/6 [==============================] - 4s 667ms/step - loss: 9.7140e-04 - accuracy: 1.0000 - val_loss: 3.7300e-04 - val_accuracy: 1.0000
Epoch 6/10
6/6 [==============================] - 4s 657ms/step - loss: 4.9362e-04 - accuracy: 1.0000 - val_loss: 2.4317e-04 - val_accuracy: 1.0000
Epoch 7/10
6/6 [==============================] - 4s 655ms/step - loss: 4.4594e-04 - accuracy: 1.0000 - val_loss: 1.7732e-04 - val_accuracy

In [10]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_pad, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

2/2 [==============================] - 1s 176ms/step - loss: 9.7902e-05 - accuracy: 1.0000
Test Loss: 9.790188050828874e-05
Test Accuracy: 1.0


In [12]:
model.save('models/model.h5')